In [3]:
pwd

'C:\\Users\\yunge\\capstone2'

In [4]:
%cd yolov5 #현재 디렉토리 yolov5 인지 확인하고 다음 코드 실행하기

C:\Users\yunge\capstone2\yolov5


In [ ]:
import cv2
import os
import numpy as np

def f_print() : 
    print('1번 : 등록')
    print('2번 : 계산')
    print('3번 : 종료')
    print('===================')
    
while 1 : 
    
    f_print()
    a = input("숫자를 입력하시오 : ")
    print()
    
    if a =='1' : 
        
        cap = cv2.VideoCapture(0)

        ######### 박스 관련 프로그래밍 ##########
        SCALE = 0.5 # 글자 크기
        THICKNESS_TEXT = 2

        width_cap = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height_cap = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        #print(width_cap) # 웹 캠 가로길이 (내 Laptop 기준:640)
        #print(height_cap) # 웹 캠 세로길이 (내 Laptop 기준:480)

        pointx_first = int(width_cap / 4) # 박스 좌상단 x
        pointy_first = int(height_cap / 4) # 박스 좌상단 y
        pointx_sec = int(3 * (width_cap / 4)) #박스 우하단 x
        pointy_sec = int(3 * (height_cap / 4)) # 박스 우하단 y

        # 박스 좌표값 return #
        #print(pointx_first, pointy_first, pointx_sec, pointy_sec)
        COLOR = (191, 255, 0) # Magenta
        THICKNESS = 3

        ######### 영상 저장 ##########
        fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        fps = cap.get(cv2.CAP_PROP_FPS) # 속도 조절은 곱하기로 가능    
        out = cv2.VideoWriter('new_ice.avi', fourcc, fps, (width_cap, height_cap))


        while cap.isOpened(): # 동영상 파일이 올바로 열렸는지?
            ret, frame = cap.read() # ret : 성공여부, frame : 받아온 이미지 (프레임)
            frame = cv2.flip(frame, 1) # 좌우대칭
            cv2.rectangle(frame, (pointx_first, pointy_first), (pointx_sec, pointy_sec), COLOR, THICKNESS) #박스 그려넣기
            cv2.putText(frame, "If you press 'q', collecting new ice images will be ended.", (pointx_first-5, pointy_first-5), 
                        cv2.FONT_HERSHEY_SIMPLEX, SCALE, COLOR, THICKNESS_TEXT)

            if not ret:
                print('더 이상 가져올 프레임이 없어요')
                break

            out.write(frame)    
            cv2.imshow('video', frame)

            if cv2.waitKey(1) == ord('q'):
                #print('사용자 입력에 의해 종료합니다')
                break       

        out.release()        
        cap.release() 
        cv2.destroyAllWindows()


        ########## 영상처리 종료 / 영상에서 프레임 자르기 ##########
        filepath = './new_ice.avi'
        video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함

        if not video.isOpened():
            print("Could not Open :", filepath)
            exit(0)

        length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # fps의 값을 작게하면 할수록 추출되는 이미지의 개수가 많아진다. 
        fps = video.get(cv2.CAP_PROP_FPS) / 4

        #print("length :", length)
        #print("width :", width)
        #print("height :", height)
        #print("fps :", fps)

        try:
            if not os.path.exists(filepath[:-4]):
                os.makedirs(filepath[:-4])
        except OSError:
            print ('Error: Creating directory. ' +  filepath[:-4])

        count = 0

        while(video.isOpened()):
            ret, image = video.read()
            if(int(video.get(1)) % fps == 0): #앞서 불러온 fps 값을 사용하여 1초마다 추출
                cv2.imwrite(filepath[:-4] + '/frame{}.jpg'.format(count), image)
        #         print('Saved frame number :', str(int(video.get(1))))
                count += 1

            if not ret:
                #print('더 이상 가져올 프레임이 없어요')
                break

        video.release()    

        ############# 이미지 처리 (박스의 크기만큼 사진 다시 만들기) ###############
        # 상자없애려면 THICKNESS 만큼 빼줄 것
        for cnt in range (0, count):
            img = cv2.imread('new_ice/frame{}.jpg'.format(cnt))
            crop = img[pointy_first:pointy_sec, pointx_first:pointx_sec]
            cv2.imwrite('new_ice/frame{}.jpg'.format(cnt), crop)


        ############# 이미지 불리기 #############
        count_origin = count
        kernel = np.ones((3,3), dtype=np.uint8)

        # 좌우대칭
        for cnt in range (0, count_origin): # image 데이터 2배
            img = cv2.imread('new_ice/frame{}.jpg'.format(cnt))
            flip_horizontal = cv2.flip(img, 1) # 좌우대칭
            cv2.imwrite('new_ice/frame{}.jpg'.format(count), flip_horizontal)
            count += 1

        # 90도 시계방향    
        for cnt in range (0, count_origin): # image 데이터 3배
            img = cv2.imread('new_ice/frame{}.jpg'.format(cnt))
            rotate_90 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            cv2.imwrite('new_ice/frame{}.jpg'.format(count), rotate_90)
            count += 1 

        # 90도 반시계방향    
        for cnt in range (0, count_origin): # image 데이터 4배
            img = cv2.imread('new_ice/frame{}.jpg'.format(cnt))
            rotate_270 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite('new_ice/frame{}.jpg'.format(count), rotate_270)
            count += 1   

        # 180도 
        for cnt in range (0, count_origin): # image 데이터 5배
            img = cv2.imread('new_ice/frame{}.jpg'.format(cnt))
            rotate_180 = cv2.rotate(img, cv2.ROTATE_180) # 180도 회전
            cv2.imwrite('new_ice/frame{}.jpg'.format(count), rotate_180)
            count += 1   

        # 가우시안 블러처리 (흐림처리1)
        for cnt in range (0, count_origin): # image 데이터 6배
            img = cv2.imread('new_ice/frame{}.jpg'.format(cnt))
            kernel_7 = cv2.GaussianBlur(img, (7, 7), 0)
            cv2.imwrite('new_ice/frame{}.jpg'.format(count), kernel_7)
            count += 1  

        # 표준편차 블러처리 (흐림처리2)
        for cnt in range (0, count_origin): # image 데이터 7배
            img = cv2.imread('new_ice/frame{}.jpg'.format(cnt))
            sigma_3 = cv2.GaussianBlur(img, (0, 0), 3)
            cv2.imwrite('new_ice/frame{}.jpg'.format(count), sigma_3)
            count += 1       

        # erode 처리 
        for cnt in range (0, count_origin): # image 데이터 8배
            img = cv2.imread('new_ice/frame{}.jpg'.format(cnt))
            erode = cv2.erode(img, kernel, iterations=3)
            cv2.imwrite('new_ice/frame{}.jpg'.format(count), erode)
            count += 1  

        # dilate 처리
        for cnt in range (0, count_origin): # image 데이터 9배
            img = cv2.imread('new_ice/frame{}.jpg'.format(cnt))
            dilate = cv2.dilate(erode, kernel, iterations=3)
            cv2.imwrite('new_ice/frame{}.jpg'.format(count), dilate)
            count += 1   
        print()
        print('*****************등록이 완료되었습니다.******************')
        print()

    elif a =='2' : 
        
        !python detect.py --weights C:\Users\yunge\capstone2\best1.pt --conf 0.4 --source 0
        print()
        print('*****************계산이 완료되었습니다.******************')
        print()
        
    elif a =='3' : 
        break

print("**************종료되었습니다.******************")

1번 : 등록
2번 : 계산
3번 : 종료
숫자를 입력하시오 : 2



detect: weights=['C:\\Users\\yunge\\capstone2\\best1.pt'], source=0, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-207-g5774a15 Python-3.7.13 torch-1.11.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7029004 parameters, 0 gradients, 15.9 GFLOPs
1/1: 0...  Success (inf frames 640x480 at 30.00 FPS)

0: 480x640 1 Togther cup, Done. (0.672s)
0: 480x640 Done. (0.666s)
0: 480x640 Done. (0.697s)
0: 480x640 1 Brabo corn, Done. (0.586s)
0: 480x640 1 Brabo corn, Done. (0.649s)

Brabo corn : 2개, 2000원
Worldc orn : 1개, 1500원
Togther cup : 1개, 6000원
총액 :  9500

***************************
계산이 완료되었습니다.
***************************

1번 : 등록
2번 : 계산
3번 : 종료



0: 480x640 1 Brabo corn, Done. (0.653s)
0: 480x640 Done. (0.628s)
0: 480x640 Done. (0.669s)
0: 480x640 Done. (0.650s)
0: 480x640 Done. (0.591s)
0: 480x640 Done. (0.727s)
0: 480x640 Done. (0.739s)
0: 480x640 Done. (0.694s)
0: 480x640 Done. (0.621s)
0: 480x640 1 Brabo corn, Done. (0.625s)
0: 480x640 1 Brabo corn, Done. (0.606s)
0: 480x640 Done. (0.623s)
0: 480x640 1 Worldc orn, Done. (0.640s)
0: 480x640 1 Worldc orn, Done. (0.621s)
0: 480x640 Done. (0.613s)
0: 480x640 Done. (0.615s)
0: 480x640 Done. (0.633s)
0: 480x640 Done. (0.710s)
0: 480x640 Done. (0.681s)
0: 480x640 Done. (0.635s)
0: 480x640 Done. (0.648s)
0: 480x640 Done. (0.640s)
0: 480x640 Done. (0.615s)
0: 480x640 Done. (0.617s)
0: 480x640 Done. (0.689s)
0: 480x640 Done. (0.633s)
0: 480x640 Done. (0.651s)
0: 480x640 Done. (0.655s)
0: 480x640 Done. (0.616s)
Speed: 2.8ms pre-process, 647.6ms inference, 0.7ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp13
